In [5]:
from __future__ import print_function
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM, Dropout
from keras.optimizers import RMSprop, adam
from keras.utils.data_utils import get_file
from keras.callbacks import History, ModelCheckpoint
import numpy as np
import pandas as pd
import random
import sys
import string

# Define number of inputs/outputs to handle
input = 16
output = 16

# Load the train data
path = "???.csv"
df = pd.read_csv(path)
print('Train total outputs:', output)

# Pre-process the data (normalize)

# cut the input in semi-redundant sequences of maxlen characters
maxlen = 60
step = 3
days_sequence = []
next_day = []
for woman_i in range(0,df.shape[2]):
    for day_i in range(0, df.shape[0] - maxlen, step):
        days_sequence.append(df.ix[day_i: day_i + maxlen,:input,woman_i])
        next_day.append(df.ix[day_i + maxlen,:input,woman_i])
print('Train nb sequences:', len(days_sequences))

X_train=days_sequence
y_train=next_day

Using TensorFlow backend.


FileNotFoundError: File b'???.csv' does not exist

In [6]:
def plot_logs(history):
    evaluation_cost = history.history['val_loss']
    evaluation_accuracy = history.history['val_acc']
    training_cost = history.history['loss']
    training_accuracy = history.history['acc']
    f, (ax1, ax2) = plt.subplots(1, 2)
    f.set_figwidth(10)
    ax1.plot(evaluation_cost,label= 'test')
    ax1.plot(training_cost, label='train')
    ax1.set_title('Cost')
    ax1.legend()
    ax2.plot(evaluation_accuracy, label='test')
    ax2.plot(training_accuracy, label='train')
    ax2.set_title('Accuracy')
    ax2.legend(loc='lower right')


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# Model params
batch_size = 128
nb_epoch = 50
np.random.seed(131078)

# build the model
print('Build model...')

filepath="lstm_1_layer.hdf5"
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, output))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

"""
filepath="lstm_2_layers.hdf5"
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, output, return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(128))
model.add(Dropout(0.5))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
"""


optimizer = adam()
model.compile(loss='binary_crossentropy', optimizer=optimizer)

# Define callback to save model
save_snapshots = ModelCheckpoint(filepath,
                                 monitor='accuracy',
                                 save_best_only=True,
                                 save_weights_only=True,
                                 mode='max',
                                 verbose=2)
callbacks_list = [save_snapshots]

history = model.fit(X_train,
                    y_train,
                    batch_size=batch_size,
                    nb_epoch=nb_epoch,
                    validation_data=(X_test, Y_test),
                    callbacks=callbacks_list,
                    verbose=2)          

score = model_.evaluate(X_test, Y_test, verbose=2)
print('Test score:', score[0])
print('Test accuracy:', score[1])  

plot_logs(history)          
          
          
          
          
          


In [ ]:
def generate_prediction(history,days=28, maxlen=60, input=16, output=16, diversity=1):
    generated = []
    x = history[-60:-1,:input]
        for i in range(day):
        
        preds = model.predict(x, verbose=0)[0]
        next_symptoms = sample(preds, diversity)
        print(next_symptoms)
        
        generated.append(next_symptoms)
        x = x[1:].append(next_symptoms)
        
        return generated
